In [2]:
import pandas as pd
from scipy.stats import zscore
import os

# Пути к файлам
combined_scores_path = r"../../../depmap_datasets/prepared_datasets/depscores.csv"
merged_data_path = r"../../../depmap_datasets/prepared_datasets/cancer_cell_lines_merged_data.csv"

# Папка для сохранения результатов
output_folder = r"./mean_gene_z-score_depscore_data"
os.makedirs(output_folder, exist_ok=True)

# Чтение данных
df_combined = pd.read_csv(combined_scores_path)
df_merged = pd.read_csv(merged_data_path)

# Выбранные типы рака
selected_types = ["AML", "ALL", "Neuroblastoma"]

# Фильтрация валидных клеточных линий
valid_diseases = df_merged['primary_disease_renamed'].value_counts().index.tolist()
df_valid_lines = df_combined[df_combined['DepMap_ID'].isin(
    df_merged[df_merged['primary_disease_renamed'].isin(valid_diseases)]['ModelID']
)]

# Добавление информации о типе рака
df_valid_lines = pd.merge(
    df_valid_lines,
    df_merged[['ModelID', 'primary_disease_renamed']],
    left_on='DepMap_ID',
    right_on='ModelID',
    how='left'
)

# Определение числовых столбцов (гены)
gene_columns = df_valid_lines.columns[1:-2]  # Все числовые столбцы кроме ID и disease

# Вычисление ГЛОБАЛЬНОГО Z-score для всех генов
df_zscore = df_valid_lines.copy()
df_zscore[gene_columns] = df_valid_lines[gene_columns].apply(zscore, nan_policy='omit')

# Фильтрация по выбранным типам рака
df_filtered = df_valid_lines[df_valid_lines["primary_disease_renamed"].isin(selected_types)]
df_zscore_filtered = df_zscore[df_zscore["primary_disease_renamed"].isin(selected_types)]

# Для каждого типа рака создаем файл с тремя столбцами
for cancer_type in selected_types:
    # Фильтруем данные по типу рака
    df_cancer = df_filtered[df_filtered["primary_disease_renamed"] == cancer_type]
    df_zscore_cancer = df_zscore_filtered[df_zscore_filtered["primary_disease_renamed"] == cancer_type]
    
    # Вычисляем средние значения
    mean_depscore = df_cancer[gene_columns].mean()
    mean_zscore = df_zscore_cancer[gene_columns].mean()
    
    # Создаем итоговый DataFrame
    result_df = pd.DataFrame({
        'Gene': gene_columns,
        'Mean_DepScore': mean_depscore.values,
        'Mean_Zscore': mean_zscore.values
    })
    
    # Сортируем по Z-score (от наиболее отрицательных)
    result_df = result_df.sort_values('Mean_Zscore')
    
    # Сохраняем в файл
    safe_name = cancer_type.replace("/", "_").lower()
    output_path = os.path.join(output_folder, f"mean_gene_z-score_depscore_{safe_name}.csv")
    result_df.to_csv(output_path, index=False)
    print(f"Файл сохранён: {output_path}")

print("Все файлы успешно созданы!")

Файл сохранён: ./mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_aml.csv
Файл сохранён: ./mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_all.csv
Файл сохранён: ./mean_gene_z-score_depscore_data\mean_gene_z-score_depscore_neuroblastoma.csv
Все файлы успешно созданы!
